In [1]:
# TODO:
#    1. early stopping not working optuna experiment

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import pandas as pd
import polars as pl
import duckdb

import boto3



import torch


from darts.models import (
    NaiveEnsembleModel,
)





import warnings
warnings.filterwarnings("ignore")

from dotenv import load_dotenv
load_dotenv()

# logging
import logging

# define log
logging.basicConfig(level=logging.INFO)
log = logging.getLogger(__name__)

/home/justinfields/Documents/github/spp_weis_price_forecast/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# https://github.com/Lightning-AI/pytorch-lightning/issues/3431
logging.getLogger("lightning.pytorch.utilities.rank_zero").setLevel(logging.WARNING)
logging.getLogger("pytorch_lightning.utilities.rank_zero").setLevel(logging.WARNING)

In [5]:
os.chdir('../..')

In [6]:
# custom modules
import src.data_engineering as de
from src import parameters
from src import plotting
from src.modeling import (
    build_fit_tsmixerx, build_fit_tide, build_fit_tft
)

## will be loaded from root when deployed

INFO:src.data_engineering:adding module path
INFO:src.modeling:adding module path


In [7]:
log.info(f'FORECAST_HORIZON: {parameters.FORECAST_HORIZON}')
log.info(f'INPUT_CHUNK_LENGTH: {parameters.INPUT_CHUNK_LENGTH}')

INFO:__main__:FORECAST_HORIZON: 120
INFO:__main__:INPUT_CHUNK_LENGTH: 168


In [8]:
torch.set_float32_matmul_precision('medium')

In [9]:
# optuna.delete_study(study_name="spp_weis_tide", storage="sqlite:///spp_trials.db")

## Data prep

In [10]:
# client for uploading model weights
s3 = boto3.client('s3')

INFO:botocore.credentials:Found credentials in environment variables.


In [11]:
# connect to database
con = de.create_database()
# log.info('getting lmp data from s3')
# con.execute("CREATE TABLE lmp AS SELECT * FROM 's3://spp-weis/data/lmp.parquet'")
# log.info('getting mtrf data from s3')
# con.execute("CREATE TABLE mtrf AS SELECT * FROM 's3://spp-weis/data/mtrf.parquet'")
# log.info('getting mtlf data from s3')
# con.execute("CREATE TABLE mtlf AS SELECT * FROM 's3://spp-weis/data/mtlf.parquet'")
# log.info('finished getting data from s3')

con.execute("SHOW TABLES").fetchall()

INFO:src.data_engineering:AWS_S3_BUCKET = 'databricks-storage-7474645306723306'
INFO:src.data_engineering:AWS_S3_FOLDER = 'unity-catalog/7474645306723306/spp-weis/'
INFO:src.data_engineering:loading lmp from s3://databricks-storage-7474645306723306/unity-catalog/7474645306723306/spp-weis/data/lmp.parquet
INFO:src.data_engineering:loading mtrf from s3://databricks-storage-7474645306723306/unity-catalog/7474645306723306/spp-weis/data/mtrf.parquet
INFO:src.data_engineering:loading mtlf from s3://databricks-storage-7474645306723306/unity-catalog/7474645306723306/spp-weis/data/mtlf.parquet


[('lmp',), ('mtlf',), ('mtrf',)]

In [1]:
# lmp = con.execute("SELECT * FROM lmp").pl()
# lmp

In [21]:
lmp = de.prep_lmp(con)
lmp

unique_id,timestamp_mst,LMP,lmp_diff
str,datetime[μs],f32,f32
"""PSCO_BHCE""",2025-01-26 07:00:00,23.717691,null
"""PSCO_BHCE""",2025-01-26 08:00:00,32.89555,9.177858
"""PSCO_BHCE""",2025-01-26 09:00:00,34.497993,1.602444
"""PSCO_BHCE""",2025-01-26 10:00:00,30.449133,-4.048861
"""PSCO_BHCE""",2025-01-26 11:00:00,24.282034,-6.167099
…,…,…,…
"""PSCO_UPIP_UPI""",2026-01-20 19:00:00,18.941059,-27.970425
"""PSCO_UPIP_UPI""",2026-01-20 20:00:00,14.689442,-4.251617
"""PSCO_UPIP_UPI""",2026-01-20 21:00:00,16.1378,1.448359


In [3]:
# lmp.select(pl.col.timestamp_mst.min())

In [23]:
# needed for plotting
lmp_df = lmp.to_pandas().rename(
    columns={
        'LMP': 'LMP_HOURLY',
        'unique_id':'node', 
        'timestamp_mst':'time'
    })

In [24]:
mtrf = de.prep_mtrf(con)
mtrf

timestamp_mst,Wind_Forecast_MW,Solar_Forecast_MW
datetime[μs],f32,f32
2025-01-26 07:00:00,1865.689941,0.24
2025-01-26 08:00:00,1825.560059,128.570007
2025-01-26 09:00:00,1195.72998,503.790009
2025-01-26 10:00:00,1004.659973,579.669983
2025-01-26 11:00:00,950.72998,972.210022
…,…,…
2026-01-31 03:00:00,1522.369995,0.0
2026-01-31 04:00:00,1527.819946,0.0
2026-01-31 05:00:00,1533.319946,0.0


In [2]:
# mtrf.select(pl.col.timestamp_mst.min())

In [26]:
mtlf = de.prep_mtlf(con)
mtlf

timestamp_mst,MTLF,Averaged_Actual
datetime[μs],f32,f32
2025-01-26 07:00:00,9268.0,9248.0
2025-01-26 08:00:00,9383.0,9392.0
2025-01-26 09:00:00,9495.0,9587.0
2025-01-26 10:00:00,9565.0,9587.0
2025-01-26 11:00:00,9389.0,9476.0
…,…,…
2026-01-31 01:00:00,7805.0,null
2026-01-31 02:00:00,7640.0,null
2026-01-31 03:00:00,7583.0,null


In [4]:
# mtlf.select(pl.col.timestamp_mst.min())

In [ ]:
all_df = de.prep_all_df(con)
all_df

INFO:src.data_engineering:preparing lmp


In [ ]:
all_df_pd = de.all_df_to_pandas(de.prep_all_df(con))
all_df_pd

In [ ]:
all_df_pd.info()

## Prep model training data

In [ ]:
lmp_all, train_all, test_all, train_test_all = de.get_train_test_all(con)

In [ ]:
train_test_all

In [ ]:
train_all

In [ ]:
all_series = de.get_series(lmp_all)
all_series[0].plot()

In [ ]:
train_test_all_series = de.get_series(train_test_all)
train_test_all_series[0].plot()

In [ ]:
train_series = de.get_series(train_all)
train_series[0].plot()

In [ ]:
test_series = de.get_series(test_all)
test_series[0].plot()

In [ ]:
futr_cov = de.get_futr_cov(all_df_pd)
futr_cov[0].plot()

In [ ]:
past_cov = de.get_past_cov(all_df_pd)
past_cov[0].plot()

In [ ]:
con.close()

## Retrain models with the best params

In [ ]:
models_tsmixer = []
if parameters.USE_TSMIXER:
    for i, param in enumerate(parameters.TSMIXER_PARAMS[:parameters.TOP_N]):
        print(f'\ni: {i} \t' + '*' * 25, flush=True)
        model_tsmixer = build_fit_tsmixerx(
            series=train_test_all_series,
            val_series=test_series,
            future_covariates=futr_cov,
            past_covariates=past_cov,
            **param
        )
        models_tsmixer += [model_tsmixer]

In [ ]:
models_tide = []
if parameters.USE_TIDE:
    for i, param in enumerate(parameters.TIDE_PARAMS[:parameters.TOP_N]):
        print(f'\ni: {i} \t' + '*' * 25, flush=True)
        model_tide = build_fit_tide(
            series=train_test_all_series,
            val_series=test_series,
            future_covariates=futr_cov,
            past_covariates=past_cov,
            **param
        )
        models_tide += [model_tide]

In [ ]:
models_tft = []
if parameters.USE_TFT:
    for i, param in enumerate(parameters.TFT_PARAMS[:parameters.TOP_N]):
        print(f'\ni: {i} \t' + '*' * 25, flush=True)
        model_tft = build_fit_tft(
            series=train_test_all_series,
            val_series=test_series,
            future_covariates=futr_cov,
            past_covariates=past_cov,
            **param
        )
        models_tft += [model_tft]

## Create ensemble model

In [ ]:
forecasting_models = models_tsmixer + models_tide + models_tft

In [ ]:
loaded_model = NaiveEnsembleModel(
    forecasting_models=forecasting_models, 
    train_forecasting_models=False
)

## Plot test predictions

In [ ]:
plot_ind = 3
plot_series = all_series[plot_ind]

In [ ]:
plot_series.static_covariates.unique_id.LMP

In [ ]:
plot_series.plot()

In [ ]:
plot_end_times = pd.date_range(
    end=test_series[plot_ind].end_time(),
    periods=10,
    freq='d',
)

plot_end_times

In [ ]:
for plot_end_time in plot_end_times:
    log.info(f'plot_end_time: {plot_end_time}')
    
    plot_node_name = plot_series.static_covariates.unique_id.LMP
    
    # if test_end_time < test_series.end_time():
    node_series = plot_series.drop_after(plot_end_time)
        
    log.info(f'plot_end_time: {plot_end_time}')
    log.info(f'node_series.end_time(): {node_series.end_time()}')
    future_cov_series = futr_cov[0]
    past_cov_series = past_cov[0]
    
    data = {
        'series': [node_series.to_json()],
        'past_covariates': [past_cov_series.to_json()],
        'future_covariates': [future_cov_series.to_json()],
        'n': parameters.FORECAST_HORIZON,
        'num_samples': 200
    }
    df = pd.DataFrame(data)
    
    plot_cov_df = future_cov_series.pd_dataframe()
    plot_cov_df = (
        plot_cov_df
        .reset_index()
        .rename(columns={'timestamp_mst':'time', 're_ratio': 'Ratio'})
    )
    
    # Predict on a Pandas DataFrame.
    df['num_samples'] = 500
    
    # for mlflow pyfunc model
    # preds_json = loaded_model.predict(df)
    # preds = TimeSeries.from_json(preds_json)

    # for darts model
    preds = loaded_model.predict(
        series=node_series,
        past_covariates=past_cov_series,
        future_covariates=future_cov_series,
        n=parameters.FORECAST_HORIZON,
        num_samples=500,
    )
    
    q_df = plotting.get_quantile_df(preds, plot_node_name)
    
    plot_df = plotting.get_mean_df(preds, plot_node_name).merge(
        plotting.get_quantile_df(preds, plot_node_name),
        left_index=True,
        right_index=True,
    )
    
    plot_df = plotting.get_plot_df(
            # TimeSeries.from_json(pred),
            preds,
            plot_cov_df,
            lmp_df,
            plot_node_name,
        )
    plot_df.rename(columns={'mean':'mean_fcast'}, inplace=True)
    plot_df
    
    plotting.plotly_forecast(plot_df, plot_node_name, show_fig=True)